In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('./movies_metadata.csv', low_memory=False, on_bad_lines='skip')
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
# 상위 2만개의 샘플을 data에 저장
data = data.head(20000)


In [5]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())


overview 열의 결측값의 수: 138


## Collection 처리

In [57]:
data['belongs_to_collection'] = data['belongs_to_collection'].fillna('{}')

def extract_dict_value(json_str, key='name'):
    """딕셔너리 문자열에서 특정 키의 값을 추출하는 범용 함수"""
    import ast
    try:
        if json_str and json_str.strip() != '{}':
            # Python 딕셔너리 문자열을 실제 딕셔너리로 변환
            dict_data = ast.literal_eval(json_str)
            # 딕셔너리인지 확인
            if isinstance(dict_data, dict):
                return dict_data.get(key, '')
            # 리스트인 경우 (예: genres, spoken_languages 등)
            elif isinstance(dict_data, list) and dict_data:
                # 첫 번째 항목이 딕셔너리인 경우
                if isinstance(dict_data[0], dict):
                    return dict_data[0].get(key, '')
            return ''
        return ''
    except Exception as e:
        print(f"Exception: {e}")
        return ''

def extract_list_dict_values(json_str, key='name', separator=' '):
    """딕셔너리 리스트에서 특정 키의 모든 값을 추출하여 합치는 함수"""
    import ast
    try:
        if json_str and json_str.strip() != '[]' and json_str.strip() != '{}':
            dict_data = ast.literal_eval(json_str)
            if isinstance(dict_data, list):
                values = [item.get(key, '') for item in dict_data if isinstance(item, dict)]
                return separator.join(filter(None, values))  # 빈 문자열 제외
            return ''
        return ''
    except Exception as e:
        print(f"Exception: {e}")
        return ''


# 다양한 컬럼에 적용
data['collection_title'] = data["belongs_to_collection"].map(lambda x: extract_list_dict_values(x, 'name'))

# genres 컬럼에서 name 추출 (리스트 형태)
data['genres_names'] = data["genres"].fillna('[]').map(lambda x: extract_list_dict_values(x, 'name'))

# spoken_languages 컬럼에서 name 추출 (리스트 형태)  
data['languages'] = data["spoken_languages"].fillna('[]').map(lambda x: extract_list_dict_values(x, 'name'))

# production_companies 컬럼에서 name 추출 (있다면)
if 'production_companies' in data.columns:
    data['companies'] = data["production_companies"].fillna('[]').map(lambda x: extract_list_dict_values(x, 'name'))

Exception: malformed node or string on line 1: <ast.Name object at 0x15147e740>


In [6]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')


In [ ]:
W_GENRES   = 2.0
W_OVERVIEW = 1.0
W_ACTORS   = 1.5
W_DIRECTOR = 1.2
W_KEYWORDS = 1.8
W_COLLECTION = 3.0

def build_weighted_text(row):
    # 리스트 필드는 공백으로 합침
    actors   = " ".join(row.get("actors", []) or [])
    keywords = " ".join(row.get("keywords", []) or [])
    genres   = row.get("genres", "") or ""
    overview = row.get("overview", "") or ""
    director = row.get("director", "") or ""
    # 단순 반복 대신, 가중치만큼 토큰을 복제해서 넣는 방식(간단하고 효과적)
    # ex) W=2.0이면 2번 반복, 1.5면 1번 반복 + 나머지는 확률적으로 더해도 되지만 여기선 반올림
    def repeat(text, w):
        n = max(1, int(round(w)))
        return (" " + text) * n if text else ""
    
    return (repeat(genres, W_GENRES) +
            repeat(overview, W_OVERVIEW) +
            repeat(actors, W_ACTORS) +
            repeat(director, W_DIRECTOR) +
            repeat(keywords, W_KEYWORDS)).strip()

data['raw_text'] = 

In [ ]:
tfidf = TfidfVectorizer(stop_words='english collection')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)


TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [8]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)


코사인 유사도 연산 결과 : (20000, 20000)


In [9]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)


4


In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]
    
    # 제목과 유사도 점수의 쌍으로 리턴 (numpy.float을 float로 변환)
    results = []
    for i, (movie_idx, score) in enumerate(sim_scores):
        title = data['title'].iloc[movie_idx]
        results.append((title, float(score)))
    
    return results

In [38]:
# get_recommendations('The Dark Knight Rises')
get_recommendations('Toy Story')


[('Toy Story 3', np.float64(0.5258229300737998)),
 ('Toy Story 2', np.float64(0.46263882186410815)),
 ('The 40 Year Old Virgin', np.float64(0.27955411504881333)),
 ('The Champ', np.float64(0.2006858906698556)),
 ('Rebel Without a Cause', np.float64(0.1827296872275654)),
 ('For Your Consideration', np.float64(0.15689199223985256)),
 ('Condorman', np.float64(0.15273436907628185)),
 ('Man on the Moon', np.float64(0.14319886672359)),
 ('Malice', np.float64(0.13751067912693812)),
 ('Factory Girl', np.float64(0.13360661610274202))]